In [1]:
%%shell
git clone https://github.com/FleshRazer/tmp.git

Cloning into 'tmp'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17 (delta 0), reused 1 (delta 0), pack-reused 16
Receiving objects: 100% (17/17), 42.34 MiB | 37.24 MiB/s, done.


In [5]:
%%shell
pip3 install --quiet \
    transformers \
    bitsandbytes \
    accelerate \
    datasets \
    peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.7 MB/s eta 0:00:00


<font size="5"><b>Restart the notebook after installing accelerate package!</b></font>  

In [8]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from peft import LoraConfig, get_peft_model

In [9]:
model_name = "NousResearch/Llama-2-7b-hf"

# Activate 4-bit precision base model loading
use_4bit = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [10]:
# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [6]:
from tmp.src.data import data_utils

dataset = data_utils.get_filtered_paranmt_hf()
dataset

Dataset({
    features: ['Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
    num_rows: 577777
})

In [13]:
dataset["reference"][0]

'If Alkar is flooding her with psychic waste, that explains the high level of neurotransmitters.'

In [16]:
prompt_template = (
    f"Detoxify the text:\n{{reference}}\n---\nDetoxified text:\n{{translation}}{{eos_token}}"
)

def apply_prompt_template(sample):
    return {
        "text": prompt_template.format(
            reference=sample["reference"],
            translation=sample["translation"],
            eos_token=tokenizer.eos_token,
        )
    }

dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))
dataset = dataset.map(lambda x: tokenizer(x["text"]), remove_columns=list(dataset.features), batched=True)
dataset = dataset.train_test_split(test_size=0.05)
dataset


Map:   0%|          | 0/577777 [00:00<?, ? examples/s]

Map:   0%|          | 0/577777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 548888
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 28889
    })
})

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    # eval_dataset=dataset["test"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        # num_train_epochs=10,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir=output_dir,
        optim="paged_adamw_8bit",
        # report_to="none",
        report_to="wandb",
        # evaluation_strategy="epoch",
        # preprocess_logits_for_metrics=None,

        save_strategy="steps",
        save_steps=25,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # compute_metrics=compute_metrics,
)